<h1><center>Airborne Infection Risk Calculator for Indoor Environments
</center></h1>

<center>By Etienne Doidic, Zhexu Li, and Nicholas Kho <center>

The global pandemic of COVID-19 has demonstrated the exceptional transmissibility of the SARS-CoV-2 virus and has highlighted the vulnerability of the built environment to airborne pathogens. For the past 40 years, Heat, Ventilation, and Air Conditioning (HVAC) systems in buildings have been optimized to recirculate conditioned air to save energy while maintaining a stable temperature [[Perez-Lombard et al. 2011]](https://doi.org/10.1016/j.enbuild.2010.10.025). However, HVAC systems operators must now consider the dangers of air recirculation and aim to prevent the buildup of airborne particles or pathogens throughout buildings in order to ensure the health of occupants and the greater public. As buildings today reopen without a vaccine it is important to understand the risk of airborne infection based on building and occupant features, for not only the current pandemic, but for future ones as well. One method that has been developed to address this problem is the infection risk calculator. These calculators allow users to input variables such as the number of occupants,  room dimensions, air change per hour (ACH), masks/no masks, etc., in order to determine the risk of infected person(s) transmitting a virus to susceptible people the room [Lelieveld et al. 2020](https://doi.org/10.3390/ijerph17218114).

While these calculators may be convenient in some cases, they are “black box” algorithms, meaning the underlying code is not accessible or transparent to users. With a large amount of skepticism surrounding the academic literature regarding this virus and its transmission [[Scheirer 2020]](https://doi.org/10.1080/00963402.2020.1778361), we believe it is important infection risk calculators are as transparent as possible. 

In this notebook we describe our methods for developing our own infection risk algorithm for the SARS-CoV-2 virus which accepts room data, and a set of parameters regarding occupants and infection rates of the surrounding community. Code and assumptions made in the algorithm will be clearly explained to users for transparency. 

In [1]:
#Imports
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
sys_path = os.path.dirname(os.getcwd())
src_path = sys_path + "/src"

sys.path.insert(1, src_path)
from calculator import *
assumptions = var
src_path = sys_path + "/notebook"

sys.path.insert(1, src_path)


Here is where users can test out different parameters in order to get different results.
It has been a challenge thus far to aquire accurate building data so we are limited to the manually inputted dataset located in notebooks/data/room_data.csv' but hopefully this gives an idea about what kind of building data are accepted by this calculator.

In the future we hope to integrate this calculator with more automated systems of data retrieval such as real-time infection rates using web-mining, and querying building data directly from building systems using Brick and Metasys, for example.

The default example calculates the infection risk of a classroom of students in Center Hall 101, a building at UC San Diego.
This cell contains the only parameters that should be changed.

In [2]:
#--------User Parameters----------------
#Number of occupants
#Room ID
room_id = 'CENTER 101'
#50 occupants is the reduced COVID-19 capacity.
n_occupants = 50 
#Duration in room in hours
t = 1.25
#Activity
#For our experiment we will assume the occupants are resting (sitting)
activities = ['resting', 'light_exercise', 'moderate_exercise', 'heavy_exercise']
activity = activities[0]
    
#Vocal Activity 
#For our experiment we will assume the occupants are speaking.
expiratory_activities = ['speaking', 'whispering', 'singing', 'breathing']
expiratory_activity = expiratory_activities[0]

Read in data regarding our subject room on the UC San Diego campus.

In [4]:
sys_path

'C:\\Users\\Etienne\\Desktop\\Infection-Risk-App'

In [3]:
room_dic = get_room_data(sys_path + '/data/raw/rm.csv', room_id)
room_dic

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Etienne\\Desktop\\Infection-Risk-App/data/raw/rm.csv'

To calculate the risk of infection in a given room, the emmission rate of quanta is needed. A quantum is defined as the viral load i.e. the dose of contaminated airborne droplet nuclei required to cause infection in 63% of persons. 
The emmission rate of quanta is caclulated with the following formula:
![title](images/formula1.1.png)

[Morawska et al. 2020](https://doi.org/10.1016/j.envint.2020.105794)

<b>cv</b> corresponds to the <b>viral load in the sputum</b> (i.e. infectious mucus) of the infected person, expressed in <b>RNA copies per mililiter</b>. The viral load in sputum of an infected person varies and is dependent on the stage of infection [[Yu et al. 2020]](https://doi.org/10.1186/s13054-020-02893-8).
A range of 1e5 - 1e9 RNA copies per mililiter for viral load in the sputum is determined but for this experiment we will choose the maximum viral load to simulate the worst case scenario for infection spread [[Walsh et al. 2020]](https://doi.org/10.1016/j.jinf.2020.06.067).

<b>ci</b> is a conversion factor defined as the ratio between one infectious quantum and the infectious dose expressed in viral RNA copies, expressed in <b>quanta per RNA copies</b>. A survey of ci's indicates a range of .01 to .1 but finally a ci of .02 was selected to reflect the average value of the infectious doses reported in [Watanabe et al. 2010](https://onlinelibrary.wiley.com/doi/full/10.1111/j.1539-6924.2010.01427.x). The study covers SARS-CoV (not to be confused with SARS-CoV-2, the virus that causes COVID-19) but was selected because it provides a more well-studied report of the infectivity of coronaviruses, which has not possible for SARS-CoV-2 due to its very recent development.

<b>Inhalation Rate (IR)</b> is the product of breathing rate and tidal volume, expressed in <b>cubic meters per hour</b>. IR is dependent on the activity of the infected subject. The following values were chosen from [Adams 1993](https://ww2.arb.ca.gov/sites/default/files/classic//research/apr/past/a033-205.pdf)


In [ ]:
pd.Series(assumptions['IR'])

<b>Ni</b> is the droplet concentrations by diameter in <b>particles per cubic centimeter</b>. The droplet concentrations are dependent on the expiratory activity of the infected person. Droplet concentrations by diameter were taken from [Morawska et al. 2009](https://doi.org/10.1016/j.jaerosci.2008.11.002). "Speaking" was considered to be the average of "singing" and "counting." 

In [ ]:
pd.DataFrame(assumptions['droplet_conc'])

<b>Vi</b> simply represents the volume of the sphere created by the droplets with the specified diameter in <b>cubic micrometers</b>

Finally, <b>j</b> refers to expiratory activity the infected person is engaged in, e.g. singing.

In [ ]:
#Calculate emission rates
ERq1 = get_quanta_emmission_rate('heavy_exercise', 'singing',assumptions)
ERq2 = get_quanta_emmission_rate('resting', 'whispering', assumptions)
print('Emission rate in quanta per hour for an infected sprinting singer: ' + str(ERq1))
print('                                 for an infected cheating student: ' + str(ERq2))

Next we must calculate the quanta concentration as a function of time using the following equation:
![title](images/formula1.png)
[Gammaitoni, Nucci 1997](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2627642/)

where <b>IVRR (hr−1)</b> represents the infectious virus removal rate in the space investigated. The infectious virus removal rate (IVRR) is the sum of three parameters [(Yang and Marr, 2011)](https://doi.org/10.1371/journal.pone.0021481): the air change per hour (<b>ACH</b>) via ventilation, the <b>particle deposition</b> on surfaces via gravitational settling, and the <b>viral inactivation rate</b> (i.e. how long the virus remains contagious in aerosol).

<b>n0</b> represents the initial number of quanta in the space

<b>I</b> is the number of infectious subjects, V is the volume of the indoor environment considered,

and <b>ERq</b> is the abovementioned quanta emission rate (quanta h−1)

This model was selected because it has been used in previous papers to estimate infection risk of other airborne diseases in indoor environments. For example [Wagner et al. 2009](https://bmcmedicine.biomedcentral.com/articles/10.1186/1741-7015-7-81)

The <b>viral inactivation rate</b> of SARS-CoV2 was determined to be .63 by [Doremalen et al. 2020](https://doi.org/10.1056/nejmc2004973).

<b>Deposition rate</b> is determined by the ratio of the settling velocity of the micrometric particles (1e-4 m/s) and 
assuming the height of the emmission source. Given the average height of a person is 1.5 m, the deposition_rate is assumed to be .24 per hour [(Chatoutsidou and Lazaridis, 2019)](https://doi.org/10.1016/j.culher.2019.02.017)

<b>ACH</b> is dependent on room conditions. Is a window open? Is the air conditioning pushing in new air? In order to determine the ACH we use the flow rate of the VAV inside of the room. VAV flow rates were determined using VAV spec sheets.

In [ ]:
ach = get_air_changes_per_hour(min(room_dic['cfm_range']), room_dic['room_volume'])
print("Min ACH for " + str(room_id) + " is " + str(ach))

To determine the number of infected occupants in a room the infection rate must be known. 

As of the 12/10/20 the infection rate for UC San Diego students is 11 cases / 11 / 1,385 tests. This information can be found on the [UCSD Return to Learn Dashboard](https://returntolearn.ucsd.edu/dashboard/index.html). The infection rate means the probability of any given student on campus to be infected is 0.79%, assuming widespread testing. 

So if our number of occupants is 50, the estimated number of infected occupants would be 0.375. When the n_occupants is too small to produce >= 1 infected occupants using the infection rate, we will assume 1 infected occupant to estimate risk  in order to produce relevant results.

In [ ]:
infection_rate = (2 + 11 + 10+ 3 + 7 + 10 + 12 + 5)/(20 + 1385 + 1375 + 286 + 1332 + 1414 + 944 + 1244)
#In order to produce a relevant result, we must assume at least one infected occupant if the  
#infection rate produces < 1 infected occupants. 
n_infected = int(infection_rate * n_occupants)
if n_infected < 1:
    n_infected = 1

<b>n0</b> or initial quanta is assumed to be zero in our experiment. This is because there doesn't seem to be an accurate way of determining this value. Assuming a natural ventilation ACH of .2, the IVRR would be 1.02(hr-1) meaning that in a little under an hour any viral particles would be inactive. So for the sake of this experiment we will assume the room has been empty for about an hour beforehand. 

In [ ]:
ivrr = .63 + .24 + ach
def quanta_concentration(t, I = n_infected, ERq = ERq1, V = room_dic['room_volume_m'], n0 = 0, IVRR = ivrr):
    return ((ERq * I) / (IVRR * V)) + (n0 + ((ERq * I) / IVRR)) * ((np.e**(-IVRR * t)) / V)

fig = plt.figure()
ax = plt.axes()
plt.xlabel('Time (hr.)')
plt.ylabel('Quanta per cubic meter')
plt.title('Minimum Ventilation')
f2 = np.vectorize(quanta_concentration) 
x = np.linspace(0, t)
ax.plot(x, f2(x));

In [ ]:
ivrr = .63 + .24
def quanta_concentration(t, I = n_infected, ERq = ERq1, V = room_dic['room_volume_m'], n0 = 0, IVRR = ivrr):
    return ((ERq * I) / (IVRR * V)) + (n0 + ((ERq * I) / IVRR)) * ((np.e**(-IVRR * t)) / V)

fig = plt.figure()
ax = plt.axes()
plt.xlabel('Time (hr.)')
plt.ylabel('Quanta per cubic meter')
plt.title('No Ventilation')
f2 = np.vectorize(quanta_concentration) 
x = np.linspace(0, t)
ax.plot(x, f2(x));

Above we have plotted the quanta concentration over time using minimum ventilation as well as no ventilation at all. 
It is important to note that this model of quanta concentration assumes that the air change will begin at t = 0 so the quanta concentration will continue to decrease over time. This implies the building HVAC system would ventilate in a constant fashion. In reality, HVAC systems tend to work in a cyclical fashion, providing short spurts of air to maintain stable temperatures. Below is an figure from Gupta et al. "Zonal Air Handling in Commercial Buildings" depicting this behavior. 

![title](images/supplyairflow.png)


Finally, to determine the the number of predicted susceptible people infected after the exposure time, we must find the product of the infection risk R and the number of susceptible people. 
To find R, we use the function below
![title](images/formula3.png)

In [ ]:

walking_talking_risk = infection_risk(t, 'CENTER 101', 50, 'light_exercise', 'speaking', sys_path + '/data/raw/rm.csv')
sitting_whisper_risk = infection_risk(t, 'CENTER 101', 50, 'resting', 'whispering', sys_path + '/data/raw/rm.csv')
average_risk = (walking_talking_risk + sitting_whisper_risk) /2

In [ ]:
infection_risk(t, 'PETER 110', 50, 'light_exercise', 'speaking', sys_path + '/data/raw/rm.csv')


We decided to average out the two actions student might engage in in the classroom, those being walking, talking,and sitting and whispering. This produced an average risk of 1.89% with 0 occupants estimated to be infected.
Lets compare this to pre-COVID capcacity...

## User Interface

In the past weeks we have worked on the user interface. Our goal is to provide an easy-to-use interface that clearly provides information about the backend process of the calculator.
This current version is a basic version to convey the function of the user interface. 
Run the code below to see the User Interface

In [ ]:
from jupyter_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash
import os
import sys
sys_path = os.path.dirname(os.getcwd())
src_path = sys_path + "/src"

sys.path.insert(1, src_path)
from calculator import *
assumptions = var
src_path = sys_path + "/notebook"
sys.path.insert(1, src_path)
rid_path = 'rm.csv'
sys_path = os.path.dirname(os.getcwd())
data_path = sys_path + "/data/"
room_df = pd.read_csv(data_path + rid_path)
rooms_id = []
for rid in room_df['Room']:
    rooms_id.append({'label': rid, 'value': rid})

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

server = app.server
activities = [{'label':'Lecture', 'value':'Lecture'}, {'label':'Studying', 'value':'Studying'}, {'label':'Singing', 'value':'Singing'}, {'label':'Social', 'value':'Social'}, {'label':'Exercising', 'value':'Exercising'}]
app.layout = html.Div([
    html.H6("Event Information"),
    #MAKE ROOM ID A DROP DOWN?
    html.Div(["RoomID: ",
              dcc.Dropdown(id='room-dropdown', value='test', options = rooms_id)]),
    html.Br(),
    html.Div(["Duration of Event (min): ",
              dcc.Input(id='time-input', value = 0, type='number')]),
    html.Br(),
    html.Div(["Number of Occupants: ",
              dcc.Input(id='occupant-input', value = 0, type='number')]),
    html.Br(),
    html.Div(["Activity: ",
              dcc.Dropdown(id='activity-dropdown', value ='test', options=activities)]),
#     html.Br(),
#     html.Button('Reset', id='reset-button'),
    html.Br(),
    html.Button('Go', id = 'go-button', n_clicks = 0),
    html.Br(),
    html.Div(id = 'calc-output', children = 'Enter values to calculate risk')

])

@app.callback(
    dash.dependencies.Output('calc-output', 'children'),
    [dash.dependencies.Input('go-button', 'n_clicks')],
    [dash.dependencies.Input('activity-dropdown', 'value')],
    [dash.dependencies.Input('room-dropdown', 'value')],
    [dash.dependencies.State('time-input', 'value')],
    [dash.dependencies.State('occupant-input', 'value')]
)
def update_calc(n_clicks, activity_dropdown, room_input, time_input, occupant_input):
    if n_clicks >= 1:
        comp_ir = ui_calc(activity_dropdown, room_input, time_input, occupant_input, data_path + rid_path)
        total_inf = int(occupant_input * comp_ir)
        to_return = 'The risk of an individual infected because of holding a(n) {} event for {} minutes in {} is {}%, given the most recent infection rates. With {} occupants, it is likely that {} occupant(s) will be infected.'.format(activity_dropdown, 
                                                                                                                                time_input, 
                                                                                                                                room_input, 
                                                                                                                                round((comp_ir * 100),2), 
                                                                                                                                occupant_input,
                                                                                                                                total_inf)
        return to_return
    else:
        return 'Enter Values to get risk calculation'

In [ ]:
app.run_server(mode="inline")

## Ongoing and Future Work

1. Unfortunately the effect of social distancing and mask wearing was not directly addressed by this model. Social distancing was indirectly addressed by the assumed 1.5m (>6ft) distance from the emission source in calculating deposition rate. The effect of masks on this model will require further research. 

2. This model measures the decay in quanta concentration over time well but it does not account for the nature of HVAC systems in buildings. In order to make this algorithm accurate and convenient it should interact more directly with the buildings it is analyzing, meaning that it queries building databases and building metadata to extract real-time and accurate data regarding dimensions, VAV flow capacity, sensor data, etc., in order to more accurately ascribe infection risk to rooms. 

3. We are currently looking into methods for extracting building data from campus databases. Building data includes room area, room heigt, and VAV CFM range by room. 
